In [1]:
#from config import (census_key, gkey)
import numpy as np
import pandas as pd
import requests
import zillow
import pprint as pp
import json

In [2]:
#----API Reference GitHub --> https://github.com/seme0021/python-zillow

api = zillow.ValuationApi()
key = "X1-ZWz17fdl35adqj_3bebs"

In [3]:
#TEST ADDRESS: 14 Hayes Ave, Elizabeth, NJ 07202

address = "501 Shumaker Dr, Monroeville, PA"
postal_code = "15146"


request_data = api.GetSearchResults(key, address, postal_code)

In [4]:
result = request_data.get_dict()
result

{'zpid': '11626766',
 'similarity_score': None,
 'links': {'home_details': 'https://www.zillow.com/homedetails/501-Shumaker-Dr-Monroeville-PA-15146/11626766_zpid/',
  'graphs_and_data': 'http://www.zillow.com/homedetails/501-Shumaker-Dr-Monroeville-PA-15146/11626766_zpid/#charts-and-data',
  'map_this_home': 'http://www.zillow.com/homes/11626766_zpid/',
  'comparables': 'http://www.zillow.com/homes/comps/11626766_zpid/'},
 'full_address': {'street': '501 Shumaker Dr',
  'zipcode': '15146',
  'city': 'Monroeville',
  'state': 'PA',
  'latitude': '40.456694',
  'longitude': '-79.763917'},
 'zestimate': {'amount': 195091,
  'amount_currency': 'USD',
  'amount_last_updated': '04/05/2020',
  'amount_change_30days': 3081,
  'valuation_range_low': 181435,
  'valuation_range_high': 216551},
 'local_realestate': {'region_name': 'Monroeville',
  'region_id': '46582',
  'region_type': 'city',
  'overview_link': 'http://www.zillow.com/local-info/PA-Monroeville/r_46582/',
  'fsbo_link': 'http://www

In [5]:
def Merge(dict1, dict2): 
    return(dict2.update(dict1)) 

In [6]:
zestimate = result['zestimate']
zestimate

{'amount': 195091,
 'amount_currency': 'USD',
 'amount_last_updated': '04/05/2020',
 'amount_change_30days': 3081,
 'valuation_range_low': 181435,
 'valuation_range_high': 216551}

In [7]:
address = result['full_address']
address

{'street': '501 Shumaker Dr',
 'zipcode': '15146',
 'city': 'Monroeville',
 'state': 'PA',
 'latitude': '40.456694',
 'longitude': '-79.763917'}

In [8]:
local = result['local_realestate']

In [9]:
merge_data = Merge(zestimate, local)
merge_data

In [10]:
test_log = pd.DataFrame.from_dict(zestimate, orient='index').T
test_log

,amount,amount_currency,amount_last_updated,amount_change_30days,valuation_range_low,valuation_range_high
0,195091,USD,04/05/2020,3081,181435,216551


In [11]:
#Function to call data from Zillow and return dataframe

def address_search(address, postal_code):
    request_data = api.GetSearchResults(key, address, postal_code)
    result = request_data.get_dict()
    address = result['full_address']
    
    zestimate = result['zestimate']
    comb_dict = dict(zestimate)
    comb_dict.update(address)
    zestimate_df = pd.DataFrame.from_dict(comb_dict, orient='index').T

    return(zestimate_df)


In [12]:
master_df = address_search("317 E 8th Ave, North Wildwood, NJ","08260")
master_df

,amount,amount_currency,amount_last_updated,amount_change_30days,valuation_range_low,valuation_range_high,street,zipcode,city,state,latitude,longitude
0,492723,USD,04/05/2020,-1942,448378,556777,317 E 8th Ave,08260,Wildwood,NJ,39.00134,-74.794258


In [19]:
#Risk adjustment calculation

master_df['risk_adj_value'] = (master_df['amount']*.85) #Risk adjusting home by 15%, reducing home val by 15%
master_df['max_cash_limit'] = master_df['risk_adj_value']*.30 #Cannot buy more than 30% of someones home equity, bc then youre pushing majority ownership
master_df

,amount,amount_currency,amount_last_updated,amount_change_30days,valuation_range_low,valuation_range_high,street,zipcode,city,state,latitude,longitude,risk_adj_val,risk_adj_value,max_cash_limit
0,492723,USD,04/05/2020,-1942,448378,556777,317 E 8th Ave,08260,Wildwood,NJ,39.00134,-74.794258,125644,418815,125644
